In [1]:
import cllaus

## Creating your own CAs

Create a class derived from `cllaus.CA`.

`__init__` needs to initialize:
 - `name` - self-explanatory
 - `next_vals` - array of vals to be toggled thru in the visualiser - the value after 10 will be on index 10; could be not an NDArray (ie. for large value sets) but has to have an [] operator and be numpy vectorizable
 - `colors` - colors corresponding to values the universe can reach - cells will be colored based on it; again has to have the [] operator, doesn't need to be vectorizable
 - `dtype` - data type of the universe to be passed to it (ca will call numpy.astype() on the current universe)

 `__call__` will step the CA, takes a NDArray as a parameter

In [5]:
import numpy as np
from numpy.typing import NDArray

# see https://en.wikipedia.org/wiki/Brian%27s_Brain
class BriansBrain(cllaus.CA):
    def __init__(self):
        self.name = "Brian's brain"
        self.next_vals = np.array([1, 2, 0], dtype=np.int8)
        self.colors = {2: "white", 1: "blue"}
        self.n_neighbours = np.zeros((100, 100), dtype=np.int8)
        self.dtype = np.int8
    
    def __call__(self, universe: NDArray[np.int8]):
        if self.n_neighbours.shape != universe.shape:
            self.n_neighbours.resize(universe.shape)
        self.n_neighbours.fill(0)

        # counting alive neighbours
        self.n_neighbours[:universe.shape[0] - 1, :] += (universe[1:, :] == 2) * 1
        self.n_neighbours[1:, :] += (universe[:universe.shape[0] - 1, :] == 2) * 1
        self.n_neighbours[:, :universe.shape[1] - 1] += (universe[:, 1:] == 2) * 1
        self.n_neighbours[:, 1:] += (universe[:, :universe.shape[1] - 1] == 2) * 1
        self.n_neighbours[:universe.shape[0] - 1, :universe.shape[1] - 1] += (universe[1:, 1:] == 2) * 1
        self.n_neighbours[1:, 1:] += (universe[:universe.shape[0] - 1, :universe.shape[1] - 1] == 2) * 1
        self.n_neighbours[:universe.shape[0] - 1, 1:] += (universe[1:, :universe.shape[1] - 1] == 2) * 1
        self.n_neighbours[1:, :universe.shape[1] - 1] += (universe[:universe.shape[0] - 1, 1:] == 2) * 1

        # dead cells become living if they have exactly 2 living neighbours
        revived = (self.n_neighbours == 2) & (universe == 0)
        universe[revived] = 2
        # dying cells die
        universe[universe == 1] = 0
        # living cells become dying
        universe[(universe == 2) & (revived == False)] = 1

In [6]:
cllaus.reset()
cllaus.rule(BriansBrain())
cllaus.paste_vals(np.array([[0, 1, 0, 0],
                            [0, 2, 2, 1],
                            [1, 2, 2, 0],
                            [0, 0, 1, 0]], dtype=np.int8), 30, 30)

cllaus.display()